In [3]:
from asyncio import futures
import pandas as pd
import plotly.express as px
from glob import glob
import json
from scipy.stats import zscore
from concurrent.futures import ThreadPoolExecutor

from src.ctgov.lib.helpers import trial_keys_query, protocol_query, _get_protocol_feature, DATA_BASE, protocol_feature_query


In [3]:
trial_keys = trial_keys_query()
keys = list(trial_keys['key'])
i = 0
for i in range(0, 50):
    n = 10000
    start = i * n
    end = start + n
    batch_keys = keys[start:end]
    protocol = protocol_query(batch_keys)
    feature = _get_protocol_feature(protocol)
    feature.to_parquet(f"{DATA_BASE}/house/parquet/{n}_{i}.parquet")
    print(i, 'complete')



,_trial_id,_org_name,_org_class,_sponsor_name,_sponsor_class,_collaborator,_condition,_location,_eligibility,_status,...,_enrollment,_phase,_location_count,_arm_count,_start_yr,_end_yr,_last_yr,_enrollment_z,_location_count_z,_arm_count_z
0,NCT04834349,M.D. Anderson Cancer Center,OTHER,M.D. Anderson Cancer Center,OTHER,None,[Recurrent Head and Neck Squamous Cell Carcino...,"[{'LocationCity': 'Houston', 'LocationContactL...",Inclusion Criteria:\n\nPatients with biopsy pr...,Recruiting,...,80.0,Phase 2,1.0,2.0,2021.0,2025.0,2021,-0.044914,-0.197093,-0.048174
1,NCT04837924,General Hospital Sveti Duh,OTHER,General Hospital Sveti Duh,OTHER,None,[Hip Fractures],"[{'LocationCity': 'Zagreb', 'LocationContactLi...",Inclusion Criteria:\n\nhip fracture after mino...,Recruiting,...,80.0,Phase 4,1.0,2.0,2021.0,2021.0,2021,-0.044914,-0.197093,-0.048174
2,NCT04838444,Valneva Austria GmbH,INDUSTRY,Valneva Austria GmbH,INDUSTRY,None,[Chikungunya Virus Infection],"[{'LocationCity': 'Phoenix', 'LocationContactL...",Inclusion Criteria:\n\nSubject participated in...,Enrolling by invitation,...,375.0,Phase 3,11.0,1.0,2021.0,2025.0,2021,0.002014,0.138348,-0.826292
3,NCT04831736,NYU Langone Health,OTHER,NYU Langone Health,OTHER,None,[Postoperative Pain],"[{'LocationCity': 'New York', 'LocationContact...","Inclusion Criteria:\n\nAdult women, aged 18 to...",Recruiting,...,30.0,Phase 4,1.0,2.0,2021.0,2022.0,2022,-0.052868,-0.197093,-0.048174
4,NCT04837820,Memorial Sloan Kettering Cancer Center,OTHER,Memorial Sloan Kettering Cancer Center,OTHER,None,[Breast Cancer],"[{'LocationCity': 'Commack', 'LocationContactL...",Inclusion Criteria:\n\nEnglish-proficient adul...,Recruiting,...,260.0,Phase 2,4.0,3.0,2021.0,2025.0,2021,-0.016280,-0.096461,0.729943
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120053,NCT00500669,Royal Hobart Hospital,OTHER_GOV,Royal Hobart Hospital,OTHER_GOV,None,"[Varicose Veins, Wound Infection, Recurrence]","[{'LocationCity': 'Hobart', 'LocationContactLi...",Inclusion Criteria:\n\nPatients undergoing pri...,Terminated,...,60.0,Phase 3,1.0,2.0,2004.0,2013.0,2017,-0.145238,-0.176074,0.024859
120054,NCT00507728,M.D. Anderson Cancer Center,OTHER,M.D. Anderson Cancer Center,OTHER,"[{'CollaboratorClass': 'NIH', 'CollaboratorNam...","[Tobacco Use Disorder, Smoking Cessation]","[{'LocationCity': 'Houston', 'LocationContactL...",Inclusion Criteria:\n\nAge: 18-65 years old\nS...,Completed,...,646.0,Phase 2,1.0,3.0,2005.0,2019.0,2021,0.173040,-0.176074,0.865731
120055,NCT00509366,Duke University,OTHER,Duke University,OTHER,"[{'CollaboratorClass': 'INDUSTRY', 'Collaborat...",[Non Small Cell Lung Cancer],"[{'LocationCity': 'Durham', 'LocationContactLi...",Inclusion Criteria:\n\nSuspected or histologic...,Terminated,...,101.0,Phase 2,8.0,4.0,2007.0,2011.0,2014,-0.122969,-0.041435,1.706604
120056,NCT00508131,"Instituto Nacional de Salud Publica, Mexico",OTHER,"Instituto Nacional de Salud Publica, Mexico",OTHER,None,"[Anemia, Iron Deficiency, Stunting]","[{'LocationCity': 'Cuernavaca', 'LocationConta...",Inclusion Criteria:\n\nHealthy children 12-30 ...,Completed,...,798.0,Phase 3,1.0,2.0,2003.0,NaN,2007,0.255596,-0.176074,0.024859


In [2]:
dfg = (
    df.groupby(["_org_class", "_org_name"])
    .agg(count=("_org_name", "count"))
    .sort_values("count", ascending=False)
    .reset_index()
)
dfg["_pct"] = dfg["count"].apply(lambda x: x / dfg["count"].sum())
dfg["_cum_pct"] = dfg["_pct"].cumsum()


px.scatter(dfg.reset_index(), x="index", y="_cum_pct", title="Org Cummulative Dist")


In [18]:
px.treemap(dfg[0:200], path=['_org_class','_org_name'], values="count", height=700, title=f"{len(df)}")

In [ ]:
px.treemap(dfg[0:1500], path=['_org_name'], values="count", height=700)

In [8]:
df['_location'].dropna().apply(lambda x: pd.json_normalize(x)['LocationZip'])
df


KeyboardInterrupt: 